## Modèle


In [1]:

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import pandas as pd


In [5]:
X = pd.read_pickle('datasets/X_all.pkl')

In [9]:
X.shape


(439, 132)

In [6]:
y = pd.read_pickle('datasets/y_all.pkl')

In [17]:
y = y.reshape((439,1))

In [18]:
y.shape

(439, 1)

## Model split


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
X_train.shape

(294, 132)

In [23]:
X_test.shape

(145, 132)

In [26]:
y_test.shape, y_train.shape


((145, 1), (294, 1))

## Model trained

In [35]:
model = Sequential([
    Dense(254, activation="relu", input_shape=(132,)),
    #Dropout(0.3),
    Dense(128, activation="relu"),
    #Dropout(0.3),
    Dense(3, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [36]:
model.fit(
    X_train,
    y_train,
    epochs=200,
    batch_size=32
)

early_stop = EarlyStopping(
    monitor="val_loss",     # ce qu'on surveille
    patience=20,             # nb d'époques sans amélioration
    restore_best_weights=True
)

Epoch 1/200


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5952 - loss: 0.9415  
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9116 - loss: 0.5187 
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9014 - loss: 0.3417 
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9422 - loss: 0.2538 
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9184 - loss: 0.2580 
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9524 - loss: 0.1905 
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9694 - loss: 0.1447 
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9762 - loss: 0.1161 
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9796 - loss: 0.1002 
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9762 - loss: 0.0956 
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9796 - loss: 0.0841 
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy:

In [49]:
pred = model.predict(X_test)

pose_id = np.argmax(pred, axis=1)



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [50]:
pose_id

array([1, 0, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 0, 0, 0, 1,
       2, 1, 0, 0, 0, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 0, 0, 1, 2, 0, 1, 0,
       0, 0, 1, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 1, 2, 1, 2,
       0, 1, 2, 0, 0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0,
       1, 1, 2, 0, 2, 2, 0, 2, 2, 0, 2, 1, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0,
       2, 0, 0, 2, 0, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 2, 0, 2, 2,
       0, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 1, 2])

In [46]:
pred


array([[1.3070854e-06, 9.9999869e-01, 2.6291081e-08],
       [9.9650627e-01, 3.3484051e-07, 3.4933286e-03],
       [1.0909649e-08, 6.1211202e-14, 1.0000000e+00],
       [4.1517919e-07, 9.9999964e-01, 5.6349485e-08],
       [2.3922067e-07, 1.7322642e-10, 9.9999976e-01]], dtype=float32)

In [41]:
X_test[0:1].shape

(1, 132)

## Model Test 

In [59]:
import cv2
import mediapipe as mp
from mediapipe import solutions
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [112]:
arbre = cv2.imread('../1_508.jpg')

In [113]:


def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

In [114]:
image_rgb = cv2.cvtColor(arbre, cv2.COLOR_BGR2RGB)

In [115]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_model:

    landmarks = get_landmarks(image_rgb,pose_model)

I0000 00:00:1766063522.378131   15209 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1766063522.380821   23147 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) HD Graphics 530 (SKL GT2)
W0000 00:00:1766063522.480706   23140 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766063522.526506   23138 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [116]:
landmarks_flatten = landmarks.flatten().reshape(1,-1)

In [117]:
landmarks_flatten.shape

(1, 132)

In [118]:
results = model.predict(landmarks_flatten)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [119]:
results

array([[1.0000000e+00, 9.7083841e-10, 2.0329416e-08]], dtype=float32)

In [120]:
np.argmax(results)

np.int64(0)